## Logistic Regression 

When we have to find sometype of real numbers like price , weight or we can say absolute numbers then we can use the Regression 


--> We also have Polynomial Regression( we'll learn later)
  Logistic regression is used for a classification problem. 
How classification problem diff from regression 
--  yes no , good , bad  where we are expecting the output out of cose and predifened number numbers or pre defined clases where prediction is not a number this come in classification. 



#### Evaluation Criteria 
1. Confusion matrix 
2. F1
3. specificity 
4. AUC -Area under curve 
5. ROC - Reciever Operator Characterstics
6. Recall 
7. Precision


In [ ]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge ,Lasso , RidgeCV , ElasticNet , ElasticNetCV, LinearRegression , LassoCV
from sklearn.metrics import accuracy_score , confusion_matrix , roc_curve, roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv")

In [ ]:
df

In [ ]:
df.describe()

label - outcome (binary result so binary class classification)



features - multi column features

In [ ]:
ProfileReport(df)

#### Pregnancies is neither standard normal distribution nor normal distribution
#### gLucose - look like normally distrbuted
#### Skin thickness - Normal distributed with some outliers
#### Isulin - little bit skewed on right side
#### Bmi - Normally Distributed
#### Diabetes Prediction -  skewed right
#### Age - Right Skewed dataset

From pearson graph we can say that their is no much multi collinearity but we can make sure it by VIF 
Their is no null value in dataset
From Histogram we can see that datset is skewed

Their are many Zeros value in dataset in different column like blood pressure  of a person cannot be zero similarly for glucose and bm for living person this may be due to device failure which is recording this value 

So we will be filling those Zeros value by something like mean or median

In [ ]:
df.columns

In [ ]:
# def TKPY_remove_zeros(df):
#   for column in df.columns:
#     print(df[column])
#     df[column] = df[column].replace(0,df[column].mean())
# TKPY_remove_zeros(df)

In [ ]:
# %matplotlib inline
# sns.pairplot(data=df)


In [ ]:
df

In [ ]:
df['BMI']=df['BMI'].replace(0,df['BMI'].mean())

In [ ]:
df['BloodPressure']=df['BloodPressure'].replace(0,df['BloodPressure'].mean())

In [ ]:
df['Insulin']=df['Insulin'].replace(0,df['Insulin'].mean())

In [ ]:
df['Glucose']=df['Glucose'].replace(0,df['Glucose'].mean())

In [ ]:
df['SkinThickness']=df['SkinThickness'].replace(0,df['SkinThickness'].mean())

In [ ]:
ProfileReport(df)

In [ ]:
# Finding Outliers in data via boxplot

In [ ]:
%matplotlib inline

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))
sns.boxplot(data=df , ax=ax)

from thise we can see insulin column has so many outliers, Bmi and skin Thickness has some outliers etc . 
we need to handle this outliers by scalling , IQR  

In [ ]:

# def TKPY_outliers_handler(df):
#     Q1 = df.quantile(0.25)
#     Q3 = df.quantile(0.75)
#     IQR = Q3 - Q1
#     l=[]
#     l=df.columns
#     df = df[~((df[l]< (Q1 - 1.5 * IQR)) |(df[l]> (Q3 + 1.5 * IQR))).any(axis=1)]
        
# TKPY_outliers_handler(df)

In [ ]:
df

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))
sns.boxplot(data=df , ax=ax)

In [ ]:
q= df['Insulin'].quantile(.95)  # to dec outliers we try decreasing dataset by some percentages trying to get low outliers

In [ ]:
df_new=df[df['Insulin']<q]

In [ ]:
df_new

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))
sns.boxplot(data=df_new , ax=ax)

In [ ]:
q= df['Pregnancies'].quantile(.98) 
df_new=df[df['Pregnancies']<q]

q= df_new['BMI'].quantile(.99) 
df_new=df[df['BMI']<q]

q= df_new['SkinThickness'].quantile(.99) 
df_new=df[df['SkinThickness']<q]

q= df_new['DiabetesPedigreeFunction'].quantile(.99) 
df_new=df[df['DiabetesPedigreeFunction']<q]

q= df_new['Age'].quantile(.98) 
df_new=df[df['Age']<q]


fig,ax=plt.subplots(figsize=(20,20))
sns.boxplot(data=df_new , ax=ax)

In [ ]:
ProfileReport(df_new)


In [ ]:
# def outlier_removal(self,data):
#     def outlier_limits(col):
#         Q3,Q1=np.nanpercentile(col,[75,25])
#         IQR=Q3-Q1
#         Ul=Q3+1.5*IQR
#         LL=Q1-1.5*IQR
#         return UL, LL
#     for column in df.columns:
#         if df[column].dtype != 'int64':
#             UL,LL = outlier_limits(df[column])
#             df[column]=np.where((df[column]>UL | (df[column]<LL), np.nan , df[column])
#     return

In [ ]:
y=df_new['Outcome']
x=df_new.drop(columns=['Outcome'])

In [ ]:
x

Our data is varying too much like in glucose their is huge difference between min and maximun value similarly for bmi  so  our data is flactuating too much we need to scale it using Standard Scaling 

In [ ]:
scaler=StandardScaler()
d=pd.DataFrame(scaler.fit_transform(df_new))
x_scales=scaler.fit_transform(x)

In [ ]:
ProfileReport(d)

now we can see in stats that min and max value are near and less flactuation

In [ ]:
fig,ax=plt.subplots(figsize=(20,20))
sns.boxplot(data=d , ax=ax)

In [ ]:
x_scales

In [ ]:
vif_df=pd.DataFrame([[x.columns[i],variance_inflation_factor(x_scales,i)] for i in range(x_scales.shape[1])],columns=["Feature","VIF_Score"])

In [ ]:
vif_df

vif score < 10 so no need to remove any column

In [ ]:
x_train,x_test, y_train,y_test=train_test_split(x_scales,y,test_size=.10 , random_state=144)

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
logr=LogisticRegression(verbose=1) # using default  solver lfbs

In [ ]:
logr.fit(x_train,y_train)

In [ ]:
logr.predict_proba([x_test[1]]) # for class 0 probab and for class 1 probab

In [ ]:
x_test[0]

In [ ]:
def TKPY_logr_predict(a,b,c,d,e,f,g,h):
    q=[a,b,c,d,e,f,g,h]
    test= scaler.fit_transform([q])
    output= logr.predict_proba(test) # for class 1 probab and for class 2  probab for the following test case
    return output
print(TKPY_logr_predict(8,183.0,64.0,20.536458,79.799479,23.3,0.672,32))

In [ ]:
logr.predict([x_test[1]]) # classes wise

In [ ]:
logr.predict_log_proba([x_test[1]])

In [ ]:
y_test.iloc[1] # checking actual value of y for predicted set

In [ ]:
logr.predict([x_test[50]])

In [ ]:
y_test.iloc[50]

In [ ]:
logr1=LogisticRegression(verbose=1, solver='liblinear') 

In [ ]:
logr1.fit(x_train,y_train)

In [ ]:
logr1.predict_proba([x_test[1]])

In [ ]:
logr1.predict([x_test[1]])

In [ ]:
logr1.predict_log_proba([x_test[1]])

In [ ]:
y_test.iloc[1]

In [ ]:
#accuracy for both model logr and logr1 


In [ ]:
y_pred1=logr1.predict(x_test)

In [ ]:
y_pred1

In [ ]:
y_pred=logr.predict(x_test)

In [ ]:
y_pred

In [ ]:
a=(confusion_matrix(y_test,y_pred1))
a

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
def model_eval(y_true,y_pred):
    tn,fp,fn,tp=confusion_matrix(y_test,y_pred).ravel()
    accuracy=(tp+tn)/(tp+fn+fp+tn)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    specificity=tn/(fp+tn)
    f1=2*(recall*precision)/(recall+precision)
    result={"Accuracy =":accuracy,"Precision=":precision,"Recall=":recall,"Specificity=":specificity,"f1_score":f1}
    return result
model_eval(y_test,y_pred1)

In [ ]:
model_eval(y_test,y_pred)

In [ ]:
roc_auc_score(y_test,y_pred)

In [ ]:
roc_curve(y_test,y_pred)

In [ ]:
auc=roc_auc_score(y_test,y_pred1)

In [ ]:
y_pred1

In [ ]:
 fpr,tpr,threshold =roc_curve(y_test,y_pred1)

In [ ]:
plt.plot(fpr,tpr,color='red',label='ROC')
plt.plot([0,1],[0,1],color='darkblue', linestyle='--',label='ROC Curve( area=%0.2f)' %auc)
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title('Receiver Operating Characteristics Curve')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

# Import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binarize the output
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(
    svm.SVC(kernel="linear", probability=True, random_state=random_state)
)
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)
lw=2
colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()


y_prob = classifier.predict_proba(X_test)

macro_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo", average="macro")
weighted_roc_auc_ovo = roc_auc_score(
    y_test, y_prob, multi_class="ovo", average="weighted"
)
macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
weighted_roc_auc_ovr = roc_auc_score(
    y_test, y_prob, multi_class="ovr", average="weighted"
)
print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)
print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
)

In [ ]:
import numpy as np
from sklearn.utils.validation import check_array
a = np.array([10,20,30,40]).reshape(1,-1)
b=check_array(a)

In [ ]:
b

In [ ]:
a

In [ ]:
a ['a']

In [ ]:
# list1 = [10, 20, 4, 45, 99]
# new_list = set(list1)
# Removing the largest element from temp list
def indices23(new_list):
    l=[]
    for i in range(0,2):
        new_list.remove(max(new_list))
        l.append(max(new_list))
    return l
print(indices23(list(range(25))))

In [ ]:
class MultiplyAdd:
    def __init__(self,m=4,c=3):
        self.m=m
        self.c=c
  
    def run(self,x):
        self.x=x
        return(self.m*self.x + self.c)
  
  
mult_add=MultiplyAdd(m=3,c=4)
mult_add.run(2)